In [2]:
source ../config_duke.sh

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Settool_Bio



In [3]:
ls ${FD_WORK}/annotation

genome  motif_cluster_jvierstra


In [4]:
ls ${FD_WORK}/annotation/motif_cluster_jvierstra

hg38_all_motifs_chroms.txt        hg38_archetype_motifs_clusters_reduce.txt
hg38_all_motifs_v1                hg38_archetype_motifs_v1
hg38_archetype_motifs_chroms.txt  motif_score_histogram.pickle


In [7]:
FDIRY=${FD_WORK}/annotation/motif_cluster_jvierstra/hg38_all_motifs_v1
FNAME=chr1.bed.gz 
zcat ${FDIRY}/${FNAME} | head

chr1	10003	10023	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10009	10029	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10015	10035	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10021	10041	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10027	10047	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10033	10053	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10039	10059	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10045	10065	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10051	10071	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC
chr1	10057	10077	RREB1_MA0073.1	3.75359698976	+	CCCTAACCCTAACCCTAACC

gzip: stdout: Broken pipe


In [9]:
FDIRY=${FD_WORK}/annotation/motif_cluster_jvierstra/hg38_all_motifs_v1
FNAME=chr1.bed.gz 
zcat ${FDIRY}/${FNAME} | head | cut -f1,2,3,5

chr1	10003	10023	3.75359698976
chr1	10009	10029	3.75359698976
chr1	10015	10035	3.75359698976
chr1	10021	10041	3.75359698976
chr1	10027	10047	3.75359698976
chr1	10033	10053	3.75359698976
chr1	10039	10059	3.75359698976
chr1	10045	10065	3.75359698976
chr1	10051	10071	3.75359698976
chr1	10057	10077	3.75359698976

gzip: stdout: Broken pipe


```
$ zcat ${FDIRY}/${FNAME} | cut -f1,2,3,5 > ${FDIRY}/test.bed
$ ls -lh ${FDIRY}/test.bed
-rw-r--r--. 1 kk319 root 6.4G Sep  2 15:37 /work/kk319/annotation/motif_cluster_jvierstra/hg38_all_motifs_v1/test.bed
$ wc -l ${FDIRY}/test.bed
181029347 /work/kk319/annotation/motif_cluster_jvierstra/hg38_all_motifs_v1/test.bed
```

In [7]:
zcat ${FDIRY}/${FNAME} | cut -f1,2,3,5 > ${FDIRY}/test.bed

In [8]:
ls -lh ${FDIRY}/test.bed

-rw-r--r--. 1 kk319 root 6.4G Sep  2 15:37 /work/kk319/annotation/motif_cluster_jvierstra/hg38_all_motifs_v1/test.bed


In [9]:
wc -l ${FDIRY}/test.bed

181029347 /work/kk319/annotation/motif_cluster_jvierstra/hg38_all_motifs_v1/test.bed


## Test singularity

In [20]:
${FD_PRJ}/sing_proj_combeffect.sh ls /home/mount/work/source/motif_cluster_jvierstra

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Settool_Bio

hg38.all_motifs.v1.0.bed.gz  hg38.archetype_motifs.v1.0.bed.gz	hub.txt


```
%%bash
FDIRY=work/source/motif_cluster_jvierstra
FNAME=hg38.archetype_motifs.v1.0.bed.gz
zcat ${FDIRY}/${FNAME} | wc -l
> 764808791
```
There are 764,808,791 (764M) lines

In [36]:
cat > motif_score_hist.py << EOF
### set env 
import numpy as np
import pandas as pd
import itertools as it
import os
import pickle

import distogram
import gzip

### start timer
import time
from datetime import timedelta
start_time = time.monotonic()

### initialize a histogram object
h = distogram.Distogram()

### stream and calculate histogram
fdiry="/home/mount/work/source/motif_cluster_jvierstra"
fname="hg38.archetype_motifs.v1.0.bed.gz"
fpath=os.path.join(fdiry, fname)

with gzip.open(fpath, "r") as f:
    
    for idx, line in enumerate(f):
    
        ### show progress
        if idx % 1000000 == 0:
            print("Line:", idx, flush=True)
        
        ### extract values
        lst = line.strip().split(b"\t")
        val = lst[4].decode("utf-8")
        val = float(val)
        
        ### update distribution
        h = distogram.update(h, val)
        
### save histogram odject as a pickle file
fdiry="/home/mount/work/annotation/motif_cluster_jvierstra"
fname="motif_score_histogram.pickle"
fpath=os.path.join(fdiry, fname)

with open(fpath, "wb") as f:
    pickle.dump(h, f)

### stop timer and show end message
end_time = time.monotonic()
print("======================================")
print("Done!")
print("Duration:")
print(timedelta(seconds=end_time - start_time))
EOF

In [37]:
### set log file directory
FD_LOG=./log

### run script using sbatch
sbatch -p all \
    --mem 8G \
    -o ${FD_LOG}/motif_score_hist.txt \
    <<'EOF'
#!/bin/bash
### set directories
source ../config_duke.sh
${FD_PRJ}/sing_proj_combeffect.sh python motif_score_hist.py
EOF

Submitted batch job 25997223


In [22]:
readlink -e ${FD_LOG}/motif_score_hist.txt

/gpfs/fs1/data/reddylab/Kuei/GitRepo/Settool_Bio/motif_cluster_jvierstra/log/motif_score_hist.txt


In [23]:
cat ${FD_LOG}/motif_score_hist.txt

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Settool_Bio

You are on Duke Server: HARDAC
BASE DIRECTORY:     /gpfs/fs1/data/reddylab/Kuei
PATH OF SOURCE:     /gpfs/fs1/data/reddylab/Kuei/source
PATH OF EXECUTABLE: /gpfs/fs1/data/reddylab/Kuei/exe
PATH OF ANNOTATION: /gpfs/fs1/data/reddylab/Kuei/annotation
PATH OF PROJECT:    /gpfs/fs1/data/reddylab/Kuei/GitRepo/Settool_Bio

[b'chr1', b'10005', b'10022', b'KLF/SP/2', b'3.7536', b'-', b'RREB1_MA0073.1', b'1']
3.7536
0
100
200
300
400
500
600
700
800
900
Traceback (most recent call last):
  File "motif_score_hist.py", line 53, in <module>
    with open(fpath, "wb") as f:
FileNotFoundError: [Errno 2] No such file or directory: './work/annotation/motif_cluster_jvierstra/motif_score_hi